In [2]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [5]:
df = pd.read_csv('covid_toy.csv')

In [6]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [11]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [12]:
df['gender'].value_counts()

gender
Female    59
Male      41
Name: count, dtype: int64

In [13]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [14]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.drop('has_covid', axis=1), df['has_covid'], test_size=0.2, random_state=0)

## Aam Zindagi - Without Column Transformer

In [17]:
# Adding simple imputer to fever column
si = SimpleImputer()
x_train_fever = si.fit_transform(x_train[['fever']])

# also the test data
x_test_fever = si.transform(x_test[['fever']])

x_train_fever.shape

(80, 1)

In [ ]:
# Ordinal Encoding -> cough column
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])
x_train_cough = oe.fit_transform(x_train[['cough']])

# also the test data
x_test_cough = oe.transform(x_test[['cough']])

In [21]:
x_train_cough.shape

(80, 1)

In [22]:
# One hot encoding -> gender,city column
ohe = OneHotEncoder(drop='first',sparse_output=False)
x_train_gender_city = ohe.fit_transform(x_train[['gender','city']])

# also the test data
x_test_gender_city = ohe.fit_transform(x_test[['gender','city']])

x_train_gender_city.shape

(80, 4)

In [23]:
# Extracting Age
x_train_age = x_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
x_test_age = x_test.drop(columns=['gender','fever','cough','city']).values

x_train_age.shape

(80, 1)

In [24]:
x_train_transformed = np.concatenate([x_train_age,x_train_fever,x_train_gender_city,x_train_cough], axis=1)
# also the test data
x_test_transformed = np.concatenate([x_test_age,x_test_fever,x_test_gender_city,x_test_cough], axis=1)

x_train_transformed.shape

(80, 7)

## Mentos zindagi - with column transformers

In [25]:
from sklearn.compose import ColumnTransformer

In [ ]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild', 'Strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop='first',sparse_output=False),['gender','city'])
    ],remainder='passthrough')


In [ ]:
# Inside hyperparameter 'transformer' you speciy all the column you want to transform
# here as i want to transform 3 columns so -
# 1. specify transformer name 'tnf1' then operation on it 'SimpleImputer()' and the column '['fever']'
# 2. specify transformer name 'tnf2' then operation on it 'OrdinalEncoder(categories=[['Mild', 'Strong']])' and the column '['cough']'
# 3. specify transformer name 'tnf3' then operation on it 'OneHotEncoder(drop='first',sparse_output=False)' and the column '['gender','city']'
# In the second hyperparameter 'remainder' specify whether you want to 'drop' the remaining column or just 'passthrough' them
 

In [36]:
transformer.fit_transform(x_train)

array([[ 99.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  22.        ],
       [104.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  56.        ],
       [ 98.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  31.        ],
       [104.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  75.        ],
       [ 99.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  72.        ],
       [ 99.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  66.        ],
       [101.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  14.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  10.        ],
       [ 98.        ,   0.        ,   1.        ,   0.        ,
          1.    

In [37]:
transformer.fit_transform(x_train).shape

(80, 7)

In [39]:
transformer.transform(x_test).shape

(20, 7)